In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

from mosaicml import *
from mosaicml.constants import MLModelFlavours
# Load the data
from connector.mosaicio import MosaicioConnector
ioConnector = MosaicioConnector()

df = pd.read_csv('/data/winequality.csv')
df.head(10)

# The target variable is 'quality'.
Y = df['quality']
X = df[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates', 'alcohol']]

# Split the data into train and test data:
from sklearn.model_selection import train_test_split
import os

# split_percentage = float(os.getenv("split_percentage"))
split_percentage = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = split_percentage)
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()
# Use ML Algorithm
from sklearn.ensemble import RandomForestClassifier

my_model = RandomForestClassifier()
my_model.fit(X_train,Y_train)

Y_true = Y_test
Y_pred = my_model.predict(X_test)

@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data_list = payload
    data_array = np.asarray(data_list)
    try:
        prediction = model.predict(data_array)
    except:
        prediction = model.predict(data_array.reshape(1, -1))
    return prediction.tolist()
import requests
req = requests.Request()
req.json = {"payload":[7.3, 0.32, 0.23, 2.3, 0.066, 35.0, 70.0, 0.99588, 3.43, 0.62, 10.1]}

score(my_model,req)

register_model(my_model,
                score,
                "wine_quality_diff_name_same_model",
                "wine quality multiclass classification model made using the randomforest classifier algorithm",
                MLModelFlavours.sklearn,
                init_script="mkdir abc \\n cd abc \\n touch abc.txt",
                input_type="json",
                explain_ai=False,
                x_train=X_train, 
                y_train=Y_train,
                y_true=Y_true,  
                y_pred=Y_pred,
                prob=my_model.predict_proba,
                model_type="classification",
                feature_names=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates', 'alcohol'], 
                feature_ids=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates', 'alcohol'],
                target_names=[3, 4, 5, 6, 7, 8],
                kyd=False,
                kyd_score=False,
                x_test=X_test,
                y_test=Y_test,
               )

/opt/conda/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


FileNotFoundError: [Errno 2] No such file or directory: '/data/winequality.csv'